# **Cocoon: Organize your Data Warehouse, ready for analysis**

### **For the demo, you Need...**

1. LLM API (cost is typically <50 cents per table)
2. Data Warehouse Con (e.g., Snowflake/DuckDB...)



In [ ]:
! pip install cocoon_data

In [ ]:
from cocoon_data import *

In [ ]:
#@title  Download example table (skip this step if you have your own table)
import requests

files = {
    "orders.csv": "https://raw.githubusercontent.com/Cocoon-Data-Transformation/cocoon/main/files/orders.csv",
}

# Loop through the files dictionary
for file_name, url in files.items():
    # Send a GET request to the URL
    response = requests.get(url)

    # Check if the request was successful
    if response.status_code == 200:
        # Open file in binary write mode
        with open(file_name, "wb") as file:
            file.write(response.content)
        print(f"{file_name} downloaded successfully.")
    else:
        print(f"Failed to download {file_name}.")

orders.csv downloaded successfully.


In [ ]:
#@title Provide your LLM API (prefer claude 3.5 sonnet)

# if you use Anthropic
openai.api_type ='Anthropic'
os.environ["ANTHROPIC_API_KEY"] = ""
cocoon_llm_setting['claude_model'] = "claude-3-5-sonnet-20240620"

# # if you use Vertex AI
# openai.api_type = 'AnthropicVertex'
# cocoon_llm_setting['vertex_region'] = "us-east5"
# cocoon_llm_setting['vertex_project_id'] = ""
# cocoon_llm_setting['vertex_model'] = "claude-3-5-sonnet@20240620"

# # if you use Bedrock
# openai.api_type = 'AnthropicBedrock'
# cocoon_llm_setting['aws_access_key'] = ""
# cocoon_llm_setting['aws_secret_key'] = ""
# cocoon_llm_setting['aws_region'] = "us-east-1"
# cocoon_llm_setting['aws_model'] = "anthropic.claude-3-5-sonnet-20240620-v1:0"

# # if you use Open AI
# openai.api_key  = ''
# cocoon_llm_setting['openai_model'] = "gpt-4-turbo"

# test if LLM works
test_message = "hello"
messages = [{"role": "user", "content": test_message}]
response = call_llm_chat(messages, temperature=0.1, top_p=0.1)
print(response['choices'][0]['message']['content'])

Hello! How can I assist you today?


In [ ]:
# @title Provide Data Warehouse Con

# In-mem duckdb loads the df
con = duckdb.connect(database=':memory:')
df = pd.read_csv("orders.csv")
table_name = "orders"
df.columns = [clean_column_name(col) for col in df.columns]
con.register(table_name, df)

# # Snowflake
# con = snowflake.connector.connect(
#     # check out your url: {account}.snowflakecomputing.com
#     account="",
#     user="",
#     password="",
#     warehouse="",
#     database="",
#     schema="",
# )

In [ ]:
query_widget, cocoon_workflow = create_cocoon_workflow(con)

In [ ]:
query_widget.display()

In [ ]:
# Note that Cocoon currently can't change the previous step
# You need to rerun the 'create_cocoon_workflow(con)' above 👆
cocoon_workflow.start()